In [19]:
import pandas as pd

data_df = pd.read_csv('adult23.csv')

data_df

,URBRRL,RATCAT_A,INCTCFLG_A,IMPINCFLG_A,LANGSPECR_A,LANGSOC_A,LANGDOC_A,LANGMED_A,LANGHM_A,PPSU,...,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,HHX,POVRATTC_A
0,3,4,0,0,NaN,NaN,NaN,NaN,NaN,2,...,NaN,NaN,1,1,1,10,1,7371.139,H029691,1.01
1,4,8,0,0,NaN,NaN,NaN,NaN,NaN,2,...,NaN,NaN,1,1,1,10,1,3146.794,H028812,2.49
2,4,14,0,0,NaN,NaN,NaN,NaN,NaN,2,...,NaN,NaN,1,1,1,10,1,10807.558,H045277,6.73
3,4,10,0,0,NaN,NaN,NaN,NaN,NaN,2,...,NaN,NaN,1,1,1,10,1,4661.643,H021192,3.43
4,4,5,0,0,NaN,NaN,NaN,NaN,NaN,2,...,NaN,NaN,1,1,1,10,1,10929.554,H025576,1.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29517,4,6,0,0,NaN,NaN,NaN,NaN,2.0,101,...,NaN,NaN,1,1,12,10,1,1873.073,H023232,1.63
29518,4,14,0,0,NaN,NaN,NaN,NaN,2.0,101,...,NaN,NaN,1,1,12,10,1,4076.747,H015656,9.18
29519,4,14,0,0,NaN,NaN,NaN,NaN,2.0,101,...,NaN,NaN,1,1,12,10,1,2786.332,H073120,7.91
29520,4,12,0,0,NaN,NaN,NaN,NaN,2.0,101,...,NaN,NaN,1,1,12,10,1,4506.270,H028235,4.33


In [20]:
thresh = len(data_df) * 0.5
data_df = data_df.dropna(axis=1, thresh=thresh)

data_df.shape

(29522, 294)

In [21]:
# Defining missing‐value codes and replace them via information from NHS codebook
missing_codes = [7, 8, 9, 97, 98, 99]
to_clean = ['CHDEV_A','ANGEV_A','MIEV_A',
            'SEX_A','HYPEV_A','CHLEV_A','DIBEV_A','SMKEV_A','BMICAT_A',
            'HISPALLP_A','EDUCP_A']
for col in to_clean:
    if col in data_df.columns:
        data_df[col] = data_df[col].replace(missing_codes, pd.NA)

# Build binary heart_disease target variable
data_df['heart_disease'] = (
    (data_df['CHDEV_A'] == 1) |
    (data_df['ANGEV_A'] == 1) |
    (data_df['MIEV_A']  == 1)
).astype(int)

# Mapping key “yes/no” flags to 0/1
binary_cols = [
    'HYPEV_A',   # ever told had hypertension
    'CHLEV_A',   # ever told had high cholesterol
    'DIBEV_A',   # ever told had diabetes
    'SMKEV_A',   # ever smoked cigarettes
]
for col in binary_cols:
    data_df[col] = data_df[col].map({1:1, 2:0}).fillna(0).astype(int)

# Ecncoding sex → male flag
data_df['male'] = data_df['SEX_A'].map({1:1, 2:0}).fillna(0).astype(int)


data_df.head()


/var/folders/xh/4c67ndhd4rggdt_j_x4syjgw0000gn/T/ipykernel_61673/1325309199.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = data_df[col].replace(missing_codes, pd.NA)
/var/folders/xh/4c67ndhd4rggdt_j_x4syjgw0000gn/T/ipykernel_61673/1325309199.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['heart_disease'] = (
/var/folders/xh/4c67ndhd4rggdt_j_x4syjgw0000gn/T/ipykernel_61673/1325309199.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,URBRRL,RATCAT_A,INCTCFLG_A,IMPINCFLG_A,LANGHM_A,PPSU,PSTRAT,HISPALLP_A,RACEALLP_A,DISAB3_A,...,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,HHX,POVRATTC_A,heart_disease,male
0,3,4,0,0,NaN,2,103,3,2,1,...,1,1,1,10,1,7371.139,H029691,1.01,0,1
1,4,8,0,0,NaN,2,122,2,1,2,...,1,1,1,10,1,3146.794,H028812,2.49,1,1
2,4,14,0,0,NaN,2,122,3,2,2,...,1,1,1,10,1,10807.558,H045277,6.73,0,1
3,4,10,0,0,NaN,2,122,2,1,2,...,1,1,1,10,1,4661.643,H021192,3.43,0,0
4,4,5,0,0,NaN,2,122,2,1,2,...,1,1,1,10,1,10929.554,H025576,1.27,0,0


In [22]:
from scipy.stats import chi2_contingency, ttest_ind

cat_feats = [
    'URBRRL', 'RATCAT_A', 'IMPINCFLG_A', 'LANGHM_A',
    'PSTRAT', 'REGION', 'EDUCP_A', 'HISPALLP_A'
]
num_feats = [
    'AGEP_A', 'WEIGHTLBTC_A', 'HEIGHTTC_A',
    'URGNT12MTC_A', 'EMERG12MTC_A'
]

chi2_results = []
for col in cat_feats:
    tbl = pd.crosstab(data_df[col].fillna('Missing'), data_df['heart_disease'])
    chi2, p, dof, expected = chi2_contingency(tbl)
    chi2_results.append({
        'feature': col,
        'chi2_stat': chi2,
        'p_value': p
    })

chi2_df = pd.DataFrame(chi2_results).sort_values('p_value')

ttest_results = []
for col in num_feats:
    grp0 = data_df.loc[data_df['heart_disease']==0, col].dropna()
    grp1 = data_df.loc[data_df['heart_disease']==1, col].dropna()
    t_stat, p = ttest_ind(grp0, grp1, equal_var=False)
    ttest_results.append({
        'feature': col,
        't_stat': t_stat,
        'p_value': p
    })

t_df = pd.DataFrame(ttest_results).sort_values('p_value')

In [23]:
print("=== Categorical features ===")
print(chi2_df.to_string(index=False))

=== Categorical features ===
    feature  chi2_stat      p_value
   RATCAT_A 226.044296 5.885019e-41
    EDUCP_A 195.738442 9.164260e-39
 HISPALLP_A 182.970878 7.934985e-37
     URBRRL  92.232085 7.262276e-20
     PSTRAT 165.884727 4.448257e-14
   LANGHM_A  59.055975 1.904329e-11
     REGION  44.347145 1.273486e-09
IMPINCFLG_A   8.661152 1.315997e-02


In [24]:
print("\n=== Numerical features ===")
print(t_df.to_string(index=False))


=== Numerical features ===
     feature     t_stat      p_value
      AGEP_A -62.650716 0.000000e+00
EMERG12MTC_A -14.627029 1.123048e-46
  HEIGHTTC_A  -3.289219 1.017011e-03
WEIGHTLBTC_A  -2.462301 1.386405e-02
URGNT12MTC_A  -1.772588 7.640523e-02
